In [ ]:
!pip install -q dask

In [ ]:
from pathlib import Path
import hvplot.pandas
import pandas as pd
import xarray as xr

DATA_PATH = Path('../../data')

In [ ]:

mgb = pd.read_csv(
    DATA_PATH/'data_cumul.csv',  sep=';', usecols=['time', 'débit_mgb'], index_col='time', converters={"time": pd.to_datetime}
    )
mgb = mgb.to_xarray().rename({"débit_mgb": "mgb"})
tamsat_reg = xr.open_dataset(DATA_PATH/'tamsat_regression_benchmark.nc')
tamsat_mlp = xr.open_dataset(DATA_PATH/'mlp_with_tamsat.nc')

data = xr.merge([tamsat_reg, tamsat_mlp])
data["mgb"] = mgb["mgb"]


In [ ]:
def plot_model_comparison(data, forecast_horizon=10, width=1000, height=600):
    """Plot line comparison between model predictions, observations and MGB data for given forecast horizon."""
    t_horizon = data.sel(forecast_horizon=f"t+{forecast_horizon}")[["pred", "obs", "mgb"]].dropna(dim="time")
    to_plot = xr.merge([t_horizon[["obs", "mgb"]], t_horizon["pred"].to_dataset(dim="model")])
    to_plot = to_plot.drop_vars([c for c in to_plot.coords if not to_plot[c].dims])
    return to_plot.to_dataframe().hvplot.line(
        width=width, 
        height=height, 
        grid=True,
        title=f"Forecast Horizon: {forecast_horizon} days",
        legend='top'
    )

In [ ]:
plot_model_comparison(data)